In [1]:
import pandas as pd

## Набор данных

In [173]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"
path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [47]:
df = pd.read_csv(path_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [98]:
df.iat[1, 2]

'Two young, white males are outside near many bushes'

# Подготовка к переводу

### Исправить некоторые знаки препинания

– Первая буква предложения – заглавная.
– Удалить пробелы в начале и в конце.
– Удалить точку в конце.
– Удалить точку.
– Удалить пробел перед запятой.
– Удалить пробел перед апострофом.
– Удалить многоточие.
– Удалить вопросительный знак.
– Удалить восклицательный знак.
– Заменить двоеточие на запятую.
– Заменить точку с запятой на запятую.
– Заменить латинский апостроф на английский.

In [2]:
def fix_punctuation_marks(a_sentence):
    return a_sentence\
        .strip()\
        .capitalize()\
        .replace(" .", "")\
        .replace(".", "")\
        .replace(" ,", ",")\
        .replace(" '", "'")\
        .replace(" ...", "")\
        .replace("...", "")\
        .replace(" ?", "")\
        .replace("?", "")\
        .replace(" !", "")\
        .replace("!", "")\
        .replace(" : ", ", ")\
        .replace(" :", ", ")\
        .replace(" ; ", ", ")\
        .replace("’", "'")

### Заменить некоторые числа на текст

In [43]:
def replace_some_numbers_with_text(a_sentence):
    return a_sentence\
        .replace(" 0 ", "zero")\
        .replace(" 1 ", "one")\
        .replace(" 2 ", "two")\
        .replace(" 3 ", "three")\
        .replace(" 4 ", "four")\
        .replace(" 5 ", "five")\
        .replace(" 6 ", "six")\
        .replace(" 7 ", "seven")\
        .replace(" 8 ", "eight")\
        .replace(" 9 ", "nine")

### Заменить некоторые числа в начале предложения на текст

In [4]:
def replace_some_start_numbers_with_text(a_sentence):
    if a_sentence[1] != ' ':
        return a_sentence

    text = ""
    if a_sentence[0] == '0':
        text = "Zero"
    elif a_sentence[0] == '1':
        text = "One"
    elif a_sentence[0] == '2':
        text = "Two"
    elif a_sentence[0] == '3':
        text = "Three"
    elif a_sentence[0] == '4':
        text = "Four"
    elif a_sentence[0] == '5':
        text = "Five"
    elif a_sentence[0] == '6':
        text = "Six"
    elif a_sentence[0] == '7':
        text = "Seven"
    elif a_sentence[0] == '8':
        text = "Eight"
    elif a_sentence[0] == '9':
        text = "Nine"

    a_sentence = text + a_sentence[1:]
    return a_sentence

# ***

– Найти кавычки.
– Найти числа.
– Найти двоеточие.

In [10]:
def is_there_number(sentence):
    for i in sentence.split():
        if i.isdigit():
            return True

    return False

In [3]:
def fix_dataframe(data, out_idxs_to_delete):
    count_chars = 0

    for idx in data.index:
        sentence = fix_punctuation_marks(data.iat[idx, 2])
        sentence = replace_some_numbers_with_text(sentence)
        sentence = replace_some_start_numbers_with_text(sentence)
        data.iat[idx, 2] = sentence

        if sentence.find('"') != -1:
            out_idxs_to_delete.append(idx)
        elif is_there_number(sentence):
            out_idxs_to_delete.append(idx)
        elif sentence.find(':') != -1:
            out_idxs_to_delete.append(idx)

        count_chars += len(sentence)

    return count_chars

In [54]:
idxs_to_delete = []
chars_num = fix_dataframe(df, idxs_to_delete)
print(chars_num)

9871649


# Удалить изображения

– у которых в описании есть кавычки
– у которых в описании есть число
– у которых в описании есть двоеточие

In [7]:
df_to_delete = df.loc[idxs_to_delete, :]
images_to_delete = df_to_delete["image_name"].unique()

(672,)

In [9]:
df = df[~df.image_name.isin(images_to_delete)]
df.shape

(155555, 3)

In [11]:
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Сохранить dataframe в файл

In [97]:
df.to_csv(path_prepared_captions, sep='|', encoding='utf-8', index=False)